# wav2vec-u Common Voice Swedish - GAN training, CPU8

> "Kaggle kernels run for maximum 9 hours, this is the most recent link in the chain"

- toc: false
- branch: master
- badges: false
- hidden: true
- categories: [kaggle, wav2vec-u]


Original [here](https://www.kaggle.com/jimregan/wav2vec-u-cv-swedish-gan-cpu8)

## Preparation

In [ ]:
!cp ../input/w2vu-cvsv-checkpoints-cpu7/checkpoint_best.pt .
!cp ../input/w2vu-cvsv-checkpoints-cpu7/checkpoint_last.pt .

In [ ]:
%%capture
!conda install -c pykaldi pykaldi -y

In [ ]:
%cd /tmp

In [ ]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

In [ ]:
!git clone https://github.com/kpu/kenlm

In [ ]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [ ]:
%%capture
%cd /tmp/kenlm
!python setup.py install
%cd /tmp

In [ ]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [ ]:
%cd /tmp/fairseq/

In [ ]:
%%capture
!python setup.py install

In [ ]:
%cd /tmp/fairseq/

In [ ]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [ ]:
%%capture
!pip install editdistance

## GAN

In [ ]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.keep_last_epochs=5 \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

In [ ]:
!bash rungan.sh